In [1]:
# Load packages

import pandas as pd
import altair as alt
import numpy as np
from vega_datasets import data
import pandas_profiling


# Need to enable this to allow work with larger datasets (https://altair-viz.github.io/user_guide/faq.html)
alt.data_transformers.enable('json')
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('json')

In [2]:
url = "https://raw.githubusercontent.com/vega/vega-datasets/master/data/movies.json"

df = pd.read_json(url)

In [3]:
df.head()

,Title,US_Gross,Worldwide_Gross,US_DVD_Sales,Production_Budget,Release_Date,MPAA_Rating,Running_Time_min,Distributor,Source,Major_Genre,Creative_Type,Director,Rotten_Tomatoes_Rating,IMDB_Rating,IMDB_Votes
0,The Land Girls,146083.0,146083.0,NaN,8000000.0,Jun 12 1998,R,NaN,Gramercy,None,None,None,None,NaN,6.1,1071.0
1,"First Love, Last Rites",10876.0,10876.0,NaN,300000.0,Aug 07 1998,R,NaN,Strand,None,Drama,None,None,NaN,6.9,207.0
2,I Married a Strange Person,203134.0,203134.0,NaN,250000.0,Aug 28 1998,None,NaN,Lionsgate,None,Comedy,None,None,NaN,6.8,865.0
3,Let's Talk About Sex,373615.0,373615.0,NaN,300000.0,Sep 11 1998,None,NaN,Fine Line,None,Comedy,None,None,13.0,NaN,NaN
4,Slam,1009819.0,1087521.0,NaN,1000000.0,Oct 09 1998,R,NaN,Trimark,Original Screenplay,Drama,Contemporary Fiction,None,62.0,3.4,165.0


In [5]:
df.to_csv(r'movies_data.csv')

In [ ]:
df['Release_Year'] = pd.to_datetime(df['Release_Date'], format = '%b %d %Y').dt.year

df_raw_select = df[["Title", "US_Gross","Worldwide_Gross", "Production_Budget", "Distributor", "Release_Year"]]

df_raw_select = df_raw_select.dropna()

df_raw_select = df_raw_select.query("Release_Year <= 2010")

df_raw_select = df_raw_select.query("Release_Year >= 1980")

PB10quantile = df_raw_select["Production_Budget"].quantile(.05)

df_raw_select = df_raw_select.query("Production_Budget >= @PB10quantile")

USG01quantile = df_raw_select["US_Gross"].quantile(.01)

df_raw_select = df_raw_select.query("US_Gross >= @USG01quantile")


df_raw_select = df_raw_select.query("Title != None")

df_raw_select = df_raw_select.query("Distributor != None")

my_dist = ["20th Century Fox", "MGM", "Warner Bros.", "Sony Pictures", "Paramount Pictures", "Walt Disney Pictures", "Universal", "Sony Pictures Classics", "Sony/Columbia", "Miramax", "Lionsgate", "Dreamworks SKG"]
df_clean_select_dist = df_raw_select[df_raw_select.Distributor.isin(my_dist)]

df_clean_select_dist["Profit"] = df_clean_select_dist["US_Gross"] - df_clean_select_dist["Production_Budget"]


df_clean = df_raw_select

In [ ]:
df_clean.to_csv(r"movies_clean_data.csv")